I. test_ffdnet_ipol.py

In [1]:
import os
import argparse
import time
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.autograd import Variable
from models import FFDNet
from utils import batch_psnr, normalize, init_logger_ipol, \
				variable_to_cv2_image, remove_dataparallel_wrapper, is_rgb

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


input= r"C:\Users\dingh\Desktop\Indigo\FFDNet-Test.png"
noise_sigma=25
add_noise=False
dont_save_results=True
no_gpu=False

print(input)

C:\Users\dingh\Desktop\Indigo\FFDNet-Test.png


In [9]:
"""
Denoise an image with the FFDNet denoising method

Copyright (C) 2018, Matias Tassano <matias.tassano@parisdescartes.fr>

This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""

import os
import argparse
import time
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.autograd import Variable
from models import FFDNet
from utils import batch_psnr, normalize, init_logger_ipol, \
				variable_to_cv2_image, remove_dataparallel_wrapper, is_rgb

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


Input= r"C:\Users\dingh\Desktop\Indigo\FFDNet-Test.png" #test image
noise_sigma=25
add_noise=False
dont_save_results=True
no_gpu=False


# Normalize noises ot [0, 1]
noise_sigma /= 255

# use CUDA?
cuda = torch.cuda.is_available()

def test_ffdnet(**args):
    r"""Denoises an input image with FFDNet
    """
    #r"""C:\Users\dingh\Desktop\Indigo\FFDNet-Test.png
    #""" 
    # Init logger
    logger = init_logger_ipol()

    # Check if input exists and if it is RGB
    try:
        rgb_den = is_rgb(args["input"])

    except:
        raise Exception('Could not open the input image')

    # Open image as a CxHxW torch.Tensor
    if rgb_den:
        in_ch = 3
        model_fn = 'models/net_rgb.pth'
        imorig = cv2.imread(args['input'])
        # from HxWxC to CxHxW, RGB image
        imorig = (cv2.cvtColor(imorig, cv2.COLOR_BGR2RGB)).transpose(2, 0, 1)
    else:
        # from HxWxC to  CxHxW grayscale image (C=1)
        in_ch = 1
        model_fn = 'models/net_gray.pth'
        imorig = cv2.imread(args['input'], cv2.IMREAD_GRAYSCALE)
        imorig = np.expand_dims(imorig, 0)
    imorig = np.expand_dims(imorig, 0)

    # Handle odd sizes
    expanded_h = False
    expanded_w = False
    sh_im = imorig.shape
    if sh_im[2]%2 == 1:
        expanded_h = True
        imorig = np.concatenate((imorig, \
                imorig[:, :, -1, :][:, :, np.newaxis, :]), axis=2)

    if sh_im[3]%2 == 1:
        expanded_w = True
        imorig = np.concatenate((imorig, \
                imorig[:, :, :, -1][:, :, :, np.newaxis]), axis=3)

    imorig = normalize(imorig)
    imorig = torch.Tensor(imorig)

    # Absolute path to model file
    if "model_pth" in list(args.keys()):
        model_fn = os.path.join(args["model_pth"], model_fn)
    else:
        model_fn = os.path.join(os.getcwd(), model_fn)
   

    # model_fn = os.path.join(os.path.abspath(os.path.dirname(__file__)), \
	# 			model_fn)

    #TO DO: FIX HARD CODED IMAGE 

    # Create model
    print('Loading model ...\n')
    net = FFDNet(num_input_channels=in_ch)

    # Load saved weights
    if args['cuda']:
        print("model_fn", model_fn)
        state_dict = torch.load(model_fn)
        device_ids = [0]
        model = nn.DataParallel(net, device_ids=device_ids).cuda()
    else:
        state_dict = torch.load(model_fn, map_location='cpu')
        # CPU mode: remove the DataParallel wrapper
        state_dict = remove_dataparallel_wrapper(state_dict)
        model = net
    model.load_state_dict(state_dict)

    # Sets the model in evaluation mode (e.g. it removes BN)
    model.eval()

    # Sets data type according to CPU or GPU modes
    if args['cuda']:
        dtype = torch.cuda.FloatTensor
    else:
        dtype = torch.FloatTensor

    # Add noise
    if args['add_noise']:
        noise = torch.FloatTensor(imorig.size()).\
                normal_(mean=0, std=args['noise_sigma'])
        imnoisy = imorig + noise
    else:
        imnoisy = imorig.clone()

        # Test mode
    with torch.no_grad(): # PyTorch v0.4.0
        imorig, imnoisy = Variable(imorig.type(dtype)), \
                        Variable(imnoisy.type(dtype))
        nsigma = Variable(
                torch.FloatTensor([args['noise_sigma']]).type(dtype))

    # Measure runtime
    start_t = time.time()

    # Estimate noise and subtract it to the input image
    im_noise_estim = model(imnoisy, nsigma)
    outim = torch.clamp(imnoisy-im_noise_estim, 0., 1.)
    stop_t = time.time()

    if expanded_h:
        imorig = imorig[:, :, :-1, :]
        outim = outim[:, :, :-1, :]
        imnoisy = imnoisy[:, :, :-1, :]

    if expanded_w:
        imorig = imorig[:, :, :, :-1]
        outim = outim[:, :, :, :-1]
        imnoisy = imnoisy[:, :, :, :-1]

    # Compute PSNR and log it
    if rgb_den:
        logger.info("### RGB denoising ###")
    else:
        logger.info("### Grayscale denoising ###")
    if args['add_noise']:
        psnr = batch_psnr(outim, imorig, 1.)
        psnr_noisy = batch_psnr(imnoisy, imorig, 1.)

        logger.info("\tPSNR noisy {0:0.2f}dB".format(psnr_noisy))
        logger.info("\tPSNR denoised {0:0.2f}dB".format(psnr))
    else:
        logger.info("\tNo noise was added, cannot compute PSNR")
    logger.info("\tRuntime {0:0.4f}s".format(stop_t-start_t))

    # Compute difference
    diffout   = 2*(outim - imorig) + .5
    diffnoise = 2*(imnoisy-imorig) + .5

    # Save images
    if not args['dont_save_results']:
        noisyimg = variable_to_cv2_image(imnoisy)
        outimg = variable_to_cv2_image(outim)
        cv2.imwrite("noisy.png", noisyimg)
        cv2.imwrite("ffdnet.png", outimg)
        print("saved_image 'noisy.png' and 'ffdnet.png'")
        if args['add_noise']:
             cv2.imwrite("noisy_diff.png", variable_to_cv2_image(diffnoise))
             cv2.imwrite("ffdnet_diff.png", variable_to_cv2_image(diffout))
             print("saved_image 'noisy_diff.png' and 'ffdnet_diff.png'")

if __name__ == "__main__":
    # Parse arguments
    parser = argparse.ArgumentParser(description="FFDNet_Test")
    parser.add_argument('--add_noise', type=str, default="True")
    parser.add_argument("--input", type=str, default="", \
                        help='path to input image')
    parser.add_argument("--suffix", type=str, default="", \
                        help='suffix to add to output name')
    parser.add_argument("--noise_sigma", type=float, default=25, \
                        help='noise level used on test set')
    parser.add_argument("--dont_save_results", action='store_true', \
                        help="don't save output images")
    parser.add_argument("--no_gpu", action='store_true', \
                        help="run model on CPU")
    #parser.add_argument("model_pth", type=str, default=None)
    argspar = parser.parse_args("")
    # Normalize noises ot [0, 1]
    argspar.noise_sigma /= 255.
    
    #give the directory for model_pth
    #argspar.model_pth = r"C:\Users\dingh\Desktop\Indigo\ffdnet-pytorch"    
    
    #Define input
    argspar.input = Input 

    # String to bool
    argspar.add_noise = (argspar.add_noise.lower() == 'true')

    # use CUDA?
    argspar.cuda = not argspar.no_gpu and torch.cuda.is_available()

    print("\n### Testing FFDNet model ###")
    print("> Parameters:")
    for p, v in zip(argspar.__dict__.keys(), argspar.__dict__.values()):
        print('\t{}: {}'.format(p, v))
    print('\n')

    test_ffdnet(**vars(argspar))

    



### Testing FFDNet model ###
> Parameters:
	add_noise: True
	input: C:\Users\dingh\Desktop\Indigo\FFDNet-Test.png
	suffix: 
	noise_sigma: 0.09803921568627451
	dont_save_results: False
	no_gpu: False
	cuda: True


rgb: True
im shape: (1366, 2048, 3)
Loading model ...

model_fn c:\Users\dingh\Desktop\Indigo\ffdnet-pytorch\models/net_rgb.pth


C:\Users\dingh\AppData\Local\Temp\ipykernel_4452\1771833235.py:109: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_fn)


saved_image 'noisy.png' and 'ffdnet.png'
saved_image 'noisy_diff.png' and 'ffdnet_diff.png'


In [53]:
print(input)

C:\Users\dingh\Desktop\Indigo\FFDNet-Test.png


II. dataset.py

In [16]:
"""
Dataset related functions

Copyright (C) 2018, Matias Tassano <matias.tassano@parisdescartes.fr>

This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""
import os
import os.path
import random
import glob
import numpy as np
import cv2
import h5py
import torch
import torch.utils.data as udata
from utils import data_augmentation, normalize

def img_to_patches(img, win, stride=1):
	r"""Converts an image to an array of patches.

	Args:
		img: a numpy array containing a CxHxW RGB (C=3) or grayscale (C=1)
			image
		win: size of the output patches
		stride: int. stride
	"""
	k = 0
	endc = img.shape[0]
	endw = img.shape[1]
	endh = img.shape[2]
	patch = img[:, 0:endw-win+0+1:stride, 0:endh-win+0+1:stride]
	total_pat_num = patch.shape[1] * patch.shape[2]
	res = np.zeros([endc, win*win, total_pat_num], np.float32)
	for i in range(win):
		for j in range(win):
			patch = img[:, i:endw-win+i+1:stride, j:endh-win+j+1:stride]
			res[:, k, :] = np.array(patch[:]).reshape(endc, total_pat_num)
			k = k + 1
	return res.reshape([endc, win, win, total_pat_num])

def prepare_data(data_path, \
				 val_data_path, \
				 patch_size, \
				 stride, \
				 max_num_patches=None, \
				 aug_times=1, \
				 gray_mode=False):
	r"""Builds the training and validations datasets by scanning the
	corresponding directories for images and extracting	patches from them.

	Args:
		data_path: path containing the training image dataset
		val_data_path: path containing the validation image dataset
		patch_size: size of the patches to extract from the images
		stride: size of stride to extract patches
		stride: size of stride to extract patches
		max_num_patches: maximum number of patches to extract
		aug_times: number of times to augment the available data minus one
		gray_mode: build the databases composed of grayscale patches
	"""
	# training database
	print('> Training database')
	scales = [1, 0.9, 0.8, 0.7]
	types = ('*.bmp', '*.png')
	files = []
	for tp in types:
		files.extend(glob.glob(os.path.join(data_path, tp)))
	files.sort()

	if gray_mode:
		traindbf = 'train_gray.h5'
		valdbf = 'val_gray.h5'
	else:
		traindbf = 'train_rgb.h5'
		valdbf = 'val_rgb.h5'

	if max_num_patches is None:
		max_num_patches = 5000000
		print("\tMaximum number of patches not set")
	else:
		print("\tMaximum number of patches set to {}".format(max_num_patches))
	train_num = 0
	i = 0
	with h5py.File(traindbf, 'w') as h5f:
		while i < len(files) and train_num < max_num_patches:
			imgor = cv2.imread(files[i])
			# h, w, c = img.shape
			for sca in scales:
				img = cv2.resize(imgor, (0, 0), fx=sca, fy=sca, \
								interpolation=cv2.INTER_CUBIC)
				if not gray_mode:
					# CxHxW RGB image
					img = (cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).transpose(2, 0, 1)
				else:
					# CxHxW grayscale image (C=1)
					img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
					img = np.expand_dims(img, 0)
				img = normalize(img)
				patches = img_to_patches(img, win=patch_size, stride=stride)
				print("\tfile: %s scale %.1f # samples: %d" % \
					  (files[i], sca, patches.shape[3]*aug_times))
				for nx in range(patches.shape[3]):
					data = data_augmentation(patches[:, :, :, nx].copy(), \
							  np.random.randint(0, 7))
					h5f.create_dataset(str(train_num), data=data)
					train_num += 1
					for mx in range(aug_times-1):
						data_aug = data_augmentation(data, np.random.randint(1, 4))
						h5f.create_dataset(str(train_num)+"_aug_%d" % (mx+1), data=data_aug)
						train_num += 1
			i += 1

	# validation database
	print('\n> Validation database')
	files = []
	for tp in types:
		files.extend(glob.glob(os.path.join(val_data_path, tp)))
	files.sort()
	h5f = h5py.File(valdbf, 'w')
	val_num = 0
	for i, item in enumerate(files):
		print("\tfile: %s" % item)
		img = cv2.imread(item)
		if not gray_mode:
			# C. H. W, RGB image
			img = (cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).transpose(2, 0, 1)
		else:
			# C, H, W grayscale image (C=1)
			img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
			img = np.expand_dims(img, 0)
		img = normalize(img)
		h5f.create_dataset(str(val_num), data=img)
		val_num += 1
	h5f.close()

	print('\n> Total')
	print('\ttraining set, # samples %d' % train_num)
	print('\tvalidation set, # samples %d\n' % val_num)

class Dataset(udata.Dataset):
	r"""Implements torch.utils.data.Dataset
	"""
	def __init__(self, train=True, gray_mode=False, shuffle=False):
		super(Dataset, self).__init__()
		self.train = train
		self.gray_mode = gray_mode
		if not self.gray_mode:
			self.traindbf = 'train_rgb.h5'
			self.valdbf = 'val_rgb.h5'
		else:
			self.traindbf = 'train_gray.h5'
			self.valdbf = 'val_gray.h5'

		if self.train:
			h5f = h5py.File(self.traindbf, 'r')
		else:
			h5f = h5py.File(self.valdbf, 'r')
		self.keys = list(h5f.keys())
		if shuffle:
			random.shuffle(self.keys)
		h5f.close()

	def __len__(self):
		return len(self.keys)

	def __getitem__(self, index):
		if self.train:
			h5f = h5py.File(self.traindbf, 'r')
		else:
			h5f = h5py.File(self.valdbf, 'r')
		key = self.keys[index]
		data = np.array(h5f[key])
		h5f.close()
		return torch.Tensor(data)


III. functions.py

In [17]:
"""
Functions implementing custom NN layers

Copyright (C) 2018, Matias Tassano <matias.tassano@parisdescartes.fr>

This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""
import torch
from torch.autograd import Function, Variable

def concatenate_input_noise_map(input, noise_sigma):
	r"""Implements the first layer of FFDNet. This function returns a
	torch.autograd.Variable composed of the concatenation of the downsampled
	input image and the noise map. Each image of the batch of size CxHxW gets
	converted to an array of size 4*CxH/2xW/2. Each of the pixels of the
	non-overlapped 2x2 patches of the input image are placed in the new array
	along the first dimension.

	Args:
		input: batch containing CxHxW images
		noise_sigma: the value of the pixels of the CxH/2xW/2 noise map
	"""
	# noise_sigma is a list of length batch_size
	N, C, H, W = input.size()
	dtype = input.type()
	sca = 2
	sca2 = sca*sca
	Cout = sca2*C
	Hout = H//sca
	Wout = W//sca
	idxL = [[0, 0], [0, 1], [1, 0], [1, 1]]

	# Fill the downsampled image with zeros
	if 'cuda' in dtype:
		downsampledfeatures = torch.cuda.FloatTensor(N, Cout, Hout, Wout).fill_(0)
	else:
		downsampledfeatures = torch.FloatTensor(N, Cout, Hout, Wout).fill_(0)

	# Build the CxH/2xW/2 noise map
	noise_map = noise_sigma.view(N, 1, 1, 1).repeat(1, C, Hout, Wout)

	# Populate output
	for idx in range(sca2):
		downsampledfeatures[:, idx:Cout:sca2, :, :] = \
			input[:, :, idxL[idx][0]::sca, idxL[idx][1]::sca]

	# concatenate de-interleaved mosaic with noise map
	return torch.cat((noise_map, downsampledfeatures), 1)

class UpSampleFeaturesFunction(Function):
	r"""Extends PyTorch's modules by implementing a torch.autograd.Function.
	This class implements the forward and backward methods of the last layer
	of FFDNet. It basically performs the inverse of
	concatenate_input_noise_map(): it converts each of the images of a
	batch of size CxH/2xW/2 to images of size C/4xHxW
	"""
	@staticmethod
	def forward(ctx, input):
		N, Cin, Hin, Win = input.size()
		dtype = input.type()
		sca = 2
		sca2 = sca*sca
		Cout = Cin//sca2
		Hout = Hin*sca
		Wout = Win*sca
		idxL = [[0, 0], [0, 1], [1, 0], [1, 1]]

		assert (Cin%sca2 == 0), \
			'Invalid input dimensions: number of channels should be divisible by 4'

		result = torch.zeros((N, Cout, Hout, Wout)).type(dtype)
		for idx in range(sca2):
			result[:, :, idxL[idx][0]::sca, idxL[idx][1]::sca] = \
				input[:, idx:Cin:sca2, :, :]

		return result

	@staticmethod
	def backward(ctx, grad_output):
		N, Cg_out, Hg_out, Wg_out = grad_output.size()
		dtype = grad_output.data.type()
		sca = 2
		sca2 = sca*sca
		Cg_in = sca2*Cg_out
		Hg_in = Hg_out//sca
		Wg_in = Wg_out//sca
		idxL = [[0, 0], [0, 1], [1, 0], [1, 1]]

		# Build output
		grad_input = torch.zeros((N, Cg_in, Hg_in, Wg_in)).type(dtype)
		# Populate output
		for idx in range(sca2):
			grad_input[:, idx:Cg_in:sca2, :, :] = \
				grad_output.data[:, :, idxL[idx][0]::sca, idxL[idx][1]::sca]

		return Variable(grad_input)

# Alias functions
upsamplefeatures = UpSampleFeaturesFunction.apply


IV. models.py

In [18]:
"""
Definition of the FFDNet model and its custom layers

Copyright (C) 2018, Matias Tassano <matias.tassano@parisdescartes.fr>

This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""
import torch.nn as nn
from torch.autograd import Variable
import functions

class UpSampleFeatures(nn.Module):
	r"""Implements the last layer of FFDNet
	"""
	def __init__(self):
		super(UpSampleFeatures, self).__init__()
	def forward(self, x):
		return functions.upsamplefeatures(x)

class IntermediateDnCNN(nn.Module):
	r"""Implements the middel part of the FFDNet architecture, which
	is basically a DnCNN net
	"""
	def __init__(self, input_features, middle_features, num_conv_layers):
		super(IntermediateDnCNN, self).__init__()
		self.kernel_size = 3
		self.padding = 1
		self.input_features = input_features
		self.num_conv_layers = num_conv_layers
		self.middle_features = middle_features
		if self.input_features == 5:
			self.output_features = 4 #Grayscale image
		elif self.input_features == 15:
			self.output_features = 12 #RGB image
		else:
			raise Exception('Invalid number of input features')

		layers = []
		layers.append(nn.Conv2d(in_channels=self.input_features,\
								out_channels=self.middle_features,\
								kernel_size=self.kernel_size,\
								padding=self.padding,\
								bias=False))
		layers.append(nn.ReLU(inplace=True))
		for _ in range(self.num_conv_layers-2):
			layers.append(nn.Conv2d(in_channels=self.middle_features,\
									out_channels=self.middle_features,\
									kernel_size=self.kernel_size,\
									padding=self.padding,\
									bias=False))
			layers.append(nn.BatchNorm2d(self.middle_features))
			layers.append(nn.ReLU(inplace=True))
		layers.append(nn.Conv2d(in_channels=self.middle_features,\
								out_channels=self.output_features,\
								kernel_size=self.kernel_size,\
								padding=self.padding,\
								bias=False))
		self.itermediate_dncnn = nn.Sequential(*layers)
	def forward(self, x):
		out = self.itermediate_dncnn(x)
		return out

class FFDNet(nn.Module):
	r"""Implements the FFDNet architecture
	"""
	def __init__(self, num_input_channels):
		super(FFDNet, self).__init__()
		self.num_input_channels = num_input_channels
		if self.num_input_channels == 1:
			# Grayscale image
			self.num_feature_maps = 64
			self.num_conv_layers = 15
			self.downsampled_channels = 5
			self.output_features = 4
		elif self.num_input_channels == 3:
			# RGB image
			self.num_feature_maps = 96
			self.num_conv_layers = 12
			self.downsampled_channels = 15
			self.output_features = 12
		else:
			raise Exception('Invalid number of input features')

		self.intermediate_dncnn = IntermediateDnCNN(\
				input_features=self.downsampled_channels,\
				middle_features=self.num_feature_maps,\
				num_conv_layers=self.num_conv_layers)
		self.upsamplefeatures = UpSampleFeatures()

	def forward(self, x, noise_sigma):
		concat_noise_x = functions.concatenate_input_noise_map(\
				x.data, noise_sigma.data)
		concat_noise_x = Variable(concat_noise_x)
		h_dncnn = self.intermediate_dncnn(concat_noise_x)
		pred_noise = self.upsamplefeatures(h_dncnn)
		return pred_noise


V. prepare_patches.py

In [39]:
"""
Construction of the training and validation databases

Copyright (C) 2018, Matias Tassano <matias.tassano@parisdescartes.fr>

This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""
import argparse
from dataset import prepare_data

if __name__ == "__main__":
	parser = argparse.ArgumentParser(description=\
								  "Building the training patch database")
	parser.add_argument("--gray", action='store_true',\
						help='prepare grayscale database instead of RGB')
	# Preprocessing parameters
	parser.add_argument("--patch_size", "--p", type=int, default=44, \
					 help="Patch size")
	parser.add_argument("--stride", "--s", type=int, default=20, \
					 help="Size of stride")
	parser.add_argument("--max_number_patches", "--m", type=int, default=100, \
						help="Maximum number of patches")
	parser.add_argument("--aug_times", "--a", type=int, default=1, \
						help="How many times to perform data augmentation")
	# Dirs
	parser.add_argument("--trainset_dir", type=str, default=r"C:\Users\dingh\Desktop\Indigo\CBSD68-dataset-master\CBSD68\original_png", \
					 help='path of trainset')
	parser.add_argument("--valset_dir", type=str, default=r"C:\Users\dingh\Desktop\Indigo\Kodak-Lossless-True-Color-Image-Suite-master\PhotoCD_PCD0992", \
						 help='path of validation set')
	args = parser.parse_args("")

	if args.gray:
		if args.trainset_dir is None:
			args.trainset_dir = 'data/gray/train'
		if args.valset_dir is None:
			args.valset_dir = 'data/gray/Set12'
	else:
		if args.trainset_dir is None:
			args.trainset_dir = 'data/rgb/CImageNet_expl'
		if args.valset_dir is None:
			args.valset_dir = 'data/rgb/Kodak24'

	print("\n### Building databases ###")
	print("> Parameters:")
	for p, v in zip(args.__dict__.keys(), args.__dict__.values()):
		print('\t{}: {}'.format(p, v))
	print('\n')

	prepare_data(args.trainset_dir,\
					args.valset_dir,\
					args.patch_size,\
					args.stride,\
					args.max_number_patches,\
					aug_times=args.aug_times,\
					gray_mode=args.gray)



### Building databases ###
> Parameters:
	gray: False
	patch_size: 44
	stride: 20
	max_number_patches: 100
	aug_times: 1
	trainset_dir: C:\Users\dingh\Desktop\Indigo\CBSD68-dataset-master\CBSD68\original_png
	valset_dir: C:\Users\dingh\Desktop\Indigo\Kodak-Lossless-True-Color-Image-Suite-master\PhotoCD_PCD0992


> Training database
	Maximum number of patches set to 100
	file: C:\Users\dingh\Desktop\Indigo\CBSD68-dataset-master\CBSD68\original_png\0000.png scale 1.0 # samples: 308
	file: C:\Users\dingh\Desktop\Indigo\CBSD68-dataset-master\CBSD68\original_png\0000.png scale 0.9 # samples: 260
	file: C:\Users\dingh\Desktop\Indigo\CBSD68-dataset-master\CBSD68\original_png\0000.png scale 0.8 # samples: 198
	file: C:\Users\dingh\Desktop\Indigo\CBSD68-dataset-master\CBSD68\original_png\0000.png scale 0.7 # samples: 150

> Validation database
	file: C:\Users\dingh\Desktop\Indigo\Kodak-Lossless-True-Color-Image-Suite-master\PhotoCD_PCD0992\1.png
	file: C:\Users\dingh\Desktop\Indigo\Kodak-Lossl

VI. train.py
changed line 170 loss[0] to loss

In [20]:
"""
Trains a FFDNet model

By default, the training starts with a learning rate equal to 1e-3 (--lr).
After the number of epochs surpasses the first milestone (--milestone), the
lr gets divided by 100. Up until this point, the orthogonalization technique
described in the FFDNet paper is performed (--no_orthog to set it off).

Copyright (C) 2018, Matias Tassano <matias.tassano@parisdescartes.fr>

This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.utils as utils
from tensorboardX import SummaryWriter
from models import FFDNet
from dataset import Dataset
from utils import weights_init_kaiming, batch_psnr, init_logger, \
			svd_orthogonalization

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def main(args):
	r"""Performs the main training loop
	"""
	# Load dataset
	print('> Loading dataset ...')
	dataset_train = Dataset(train=True, gray_mode=args.gray, shuffle=True)
	dataset_val = Dataset(train=False, gray_mode=args.gray, shuffle=False)
	loader_train = DataLoader(dataset=dataset_train, num_workers=6, \
							   batch_size=args.batch_size, shuffle=True)
	print("\t# of training samples: %d\n" % int(len(dataset_train)))

	# Init loggers
	if not os.path.exists(args.log_dir):
		os.makedirs(args.log_dir)
	writer = SummaryWriter(args.log_dir)
	logger = init_logger(args)

	# Create model
	if not args.gray:
		in_ch = 3
	else:
		in_ch = 1
	net = FFDNet(num_input_channels=in_ch)
	# Initialize model with He init
	net.apply(weights_init_kaiming)
	# Define loss
	criterion = nn.MSELoss(size_average=False)

	# Move to GPU
	device_ids = [0]
	model = nn.DataParallel(net, device_ids=device_ids).cuda()
	criterion.cuda()

	# Optimizer
	optimizer = optim.Adam(model.parameters(), lr=args.lr)

	# Resume training or start anew
	if args.resume_training:
		resumef = os.path.join(args.log_dir, 'ckpt.pth')
		if os.path.isfile(resumef):
			checkpoint = torch.load(resumef)
			print("> Resuming previous training")
			model.load_state_dict(checkpoint['state_dict'])
			optimizer.load_state_dict(checkpoint['optimizer'])
			new_epoch = args.epochs
			new_milestone = args.milestone
			current_lr = args.lr
			args = checkpoint['args']
			training_params = checkpoint['training_params']
			start_epoch = training_params['start_epoch']
			args.epochs = new_epoch
			args.milestone = new_milestone
			args.lr = current_lr
			print("=> loaded checkpoint '{}' (epoch {})"\
				  .format(resumef, start_epoch))
			print("=> loaded parameters :")
			print("==> checkpoint['optimizer']['param_groups']")
			print("\t{}".format(checkpoint['optimizer']['param_groups']))
			print("==> checkpoint['training_params']")
			for k in checkpoint['training_params']:
				print("\t{}, {}".format(k, checkpoint['training_params'][k]))
			argpri = vars(checkpoint['args'])
			print("==> checkpoint['args']")
			for k in argpri:
				print("\t{}, {}".format(k, argpri[k]))

			args.resume_training = False
		else:
			raise Exception("Couldn't resume training with checkpoint {}".\
				   format(resumef))
	else:
		start_epoch = 0
		training_params = {}
		training_params['step'] = 0
		training_params['current_lr'] = 0
		training_params['no_orthog'] = args.no_orthog

	# Training
	for epoch in range(start_epoch, args.epochs):
		# Learning rate value scheduling according to args.milestone
		if epoch > args.milestone[1]:
			current_lr = args.lr / 1000.
			training_params['no_orthog'] = True
		elif epoch > args.milestone[0]:
			current_lr = args.lr / 10.
		else:
			current_lr = args.lr

		# set learning rate in optimizer
		for param_group in optimizer.param_groups:
			param_group["lr"] = current_lr
		print('learning rate %f' % current_lr)

		# train
		for i, data in enumerate(loader_train, 0):
			# Pre-training step
			model.train()
			model.zero_grad()
			optimizer.zero_grad()

			# inputs: noise and noisy image
			img_train = data
			noise = torch.zeros(img_train.size())
			stdn = np.random.uniform(args.noiseIntL[0], args.noiseIntL[1], \
							size=noise.size()[0])
			for nx in range(noise.size()[0]):
				sizen = noise[0, :, :, :].size()
				noise[nx, :, :, :] = torch.FloatTensor(sizen).\
									normal_(mean=0, std=stdn[nx])
			imgn_train = img_train + noise
			# Create input Variables
			img_train = Variable(img_train.cuda())
			imgn_train = Variable(imgn_train.cuda())
			noise = Variable(noise.cuda())
			stdn_var = Variable(torch.cuda.FloatTensor(stdn))

			# Evaluate model and optimize it
			out_train = model(imgn_train, stdn_var)
			loss = criterion(out_train, noise) / (imgn_train.size()[0]*2)
			loss.backward()
			optimizer.step()

			# Results
			model.eval()
			out_train = torch.clamp(imgn_train-model(imgn_train, stdn_var), 0., 1.)
			psnr_train = batch_psnr(out_train, img_train, 1.)
			# PyTorch v0.4.0: loss.data[0] --> loss.item()

			if training_params['step'] % args.save_every == 0:
				# Apply regularization by orthogonalizing filters
				if not training_params['no_orthog']:
					model.apply(svd_orthogonalization)

				# Log the scalar values
				writer.add_scalar('loss', loss.data, training_params['step'])
				writer.add_scalar('PSNR on training data', psnr_train, \
					  training_params['step'])
				print("[epoch %d][%d/%d] loss: %.4f PSNR_train: %.4f" %\
					(epoch+1, i+1, len(loader_train), loss.data, psnr_train))
			training_params['step'] += 1
		# The end of each epoch
		model.eval()

		# Validation
		psnr_val = 0
		for valimg in dataset_val:
			img_val = torch.unsqueeze(valimg, 0)
			noise = torch.FloatTensor(img_val.size()).\
					normal_(mean=0, std=args.val_noiseL)
			imgn_val = img_val + noise
			img_val, imgn_val = Variable(img_val.cuda()), Variable(imgn_val.cuda())
			sigma_noise = Variable(torch.cuda.FloatTensor([args.val_noiseL]))
			out_val = torch.clamp(imgn_val-model(imgn_val, sigma_noise), 0., 1.)
			psnr_val += batch_psnr(out_val, img_val, 1.)
		psnr_val /= len(dataset_val)
		print("\n[epoch %d] PSNR_val: %.4f" % (epoch+1, psnr_val))
		writer.add_scalar('PSNR on validation data', psnr_val, epoch)
		writer.add_scalar('Learning rate', current_lr, epoch)

		# Log val images
		try:
			if epoch == 0:
				# Log graph of the model
				writer.add_graph(model, (imgn_val, sigma_noise), )
				# Log validation images
				for idx in range(2):
					imclean = utils.make_grid(img_val.data[idx].clamp(0., 1.), \
											nrow=2, normalize=False, scale_each=False)
					imnsy = utils.make_grid(imgn_val.data[idx].clamp(0., 1.), \
											nrow=2, normalize=False, scale_each=False)
					writer.add_image('Clean validation image {}'.format(idx), imclean, epoch)
					writer.add_image('Noisy validation image {}'.format(idx), imnsy, epoch)
			for idx in range(2):
				imrecons = utils.make_grid(out_val.data[idx].clamp(0., 1.), \
										nrow=2, normalize=False, scale_each=False)
				writer.add_image('Reconstructed validation image {}'.format(idx), \
								imrecons, epoch)
			# Log training images
			imclean = utils.make_grid(img_train.data, nrow=8, normalize=True, \
						 scale_each=True)
			writer.add_image('Training patches', imclean, epoch)

		except Exception as e:
			logger.error("Couldn't log results: {}".format(e))

		# save model and checkpoint
		training_params['start_epoch'] = epoch + 1
		torch.save(model.state_dict(), os.path.join(args.log_dir, 'net.pth'))
		save_dict = { \
			'state_dict': model.state_dict(), \
			'optimizer' : optimizer.state_dict(), \
			'training_params': training_params, \
			'args': args\
			}
		torch.save(save_dict, os.path.join(args.log_dir, 'ckpt.pth'))
		if epoch % args.save_every_epochs == 0:
			torch.save(save_dict, os.path.join(args.log_dir, \
									  'ckpt_e{}.pth'.format(epoch+1)))
		del save_dict

if __name__ == "__main__":

	parser = argparse.ArgumentParser(description="FFDNet")
	parser.add_argument("--gray", action='store_true',\
						help='train grayscale image denoising instead of RGB')

	parser.add_argument("--log_dir", type=str, default="logs", \
					 help='path of log files')
	#Training parameters
	parser.add_argument("--batch_size", type=int, default=128, 	\
					 help="Training batch size")
	parser.add_argument("--epochs", "--e", type=int, default=80, \
					 help="Number of total training epochs")
	parser.add_argument("--resume_training", "--r", action='store_true',\
						help="resume training from a previous checkpoint")
	parser.add_argument("--milestone", nargs=2, type=int, default=[50, 60], \
						help="When to decay learning rate; should be lower than 'epochs'")
	parser.add_argument("--lr", type=float, default=1e-3, \
					 help="Initial learning rate")
	parser.add_argument("--no_orthog", action='store_true',\
						help="Don't perform orthogonalization as regularization")
	parser.add_argument("--save_every", type=int, default=10,\
						help="Number of training steps to log psnr and perform \
						orthogonalization")
	parser.add_argument("--save_every_epochs", type=int, default=5,\
						help="Number of training epochs to save state")
	parser.add_argument("--noiseIntL", nargs=2, type=int, default=[0, 75], \
					 help="Noise training interval")
	parser.add_argument("--val_noiseL", type=float, default=25, \
						help='noise level used on validation set')
	argspar = parser.parse_args("")
	# Normalize noise between [0, 1]
	argspar.val_noiseL /= 255.
	argspar.noiseIntL[0] /= 255.
	argspar.noiseIntL[1] /= 255.

	print("\n### Training FFDNet model ###")
	print("> Parameters:")
	for p, v in zip(argspar.__dict__.keys(), argspar.__dict__.values()):
		print('\t{}: {}'.format(p, v))
	print('\n')

	main(argspar)



### Training FFDNet model ###
> Parameters:
	gray: False
	log_dir: logs
	batch_size: 128
	epochs: 80
	resume_training: False
	milestone: [50, 60]
	lr: 0.001
	no_orthog: False
	save_every: 10
	save_every_epochs: 5
	noiseIntL: [0.0, 0.29411764705882354]
	val_noiseL: 0.09803921568627451


> Loading dataset ...
	# of training samples: 916

learning rate 0.001000


c:\Users\dingh\Desktop\Indigo\ffdnet-pytorch\utils.py:33: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  nn.init.kaiming_normal(lyr.weight.data, a=0, mode='fan_in')
c:\Users\dingh\Desktop\Indigo\ffdnet-pytorch\utils.py:39: FutureWarning: `nn.init.constant` is now deprecated in favor of `nn.init.constant_`.
  nn.init.constant(lyr.bias.data, 0.0)
c:\Users\dingh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


[epoch 1][1/8] loss: 77.7016 PSNR_train: 20.6819

[epoch 1] PSNR_val: 20.4358
learning rate 0.001000
[epoch 2][3/8] loss: 80.7904 PSNR_train: 19.6632

[epoch 2] PSNR_val: 20.4295
learning rate 0.001000
[epoch 3][5/8] loss: 68.9312 PSNR_train: 18.8181

[epoch 3] PSNR_val: 20.4296
learning rate 0.001000
[epoch 4][7/8] loss: 53.5521 PSNR_train: 18.8754

[epoch 4] PSNR_val: 20.4320
learning rate 0.001000

[epoch 5] PSNR_val: 20.4229
learning rate 0.001000
[epoch 6][1/8] loss: 29.1655 PSNR_train: 19.9964

[epoch 6] PSNR_val: 20.4054
learning rate 0.001000
[epoch 7][3/8] loss: 22.8981 PSNR_train: 19.6613

[epoch 7] PSNR_val: 20.4149
learning rate 0.001000
[epoch 8][5/8] loss: 20.5306 PSNR_train: 19.0550

[epoch 8] PSNR_val: 20.3945
learning rate 0.001000
[epoch 9][7/8] loss: 14.6931 PSNR_train: 18.9147

[epoch 9] PSNR_val: 20.6287
learning rate 0.001000

[epoch 10] PSNR_val: 21.4120
learning rate 0.001000
[epoch 11][1/8] loss: 13.8024 PSNR_train: 19.7802

[epoch 11] PSNR_val: 22.5402
learnin

VII. utils.py

In [ ]:
"""
Different utilities such as orthogonalization of weights, initialization of
loggers, etc

Copyright (C) 2018, Matias Tassano <matias.tassano@parisdescartes.fr>

This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""
import subprocess
import math
import logging
import numpy as np
import cv2
import torch
import torch.nn as nn
from skimage.metrics import peak_signal_noise_ratio

def weights_init_kaiming(lyr):
	r"""Initializes weights of the model according to the "He" initialization
	method described in "Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification" - He, K. et al. (2015), using a
    normal distribution.
	This function is to be called by the torch.nn.Module.apply() method,
	which applies weights_init_kaiming() to every layer of the model.
	"""
	classname = lyr.__class__.__name__
	if classname.find('Conv') != -1:
		nn.init.kaiming_normal(lyr.weight.data, a=0, mode='fan_in')
	elif classname.find('Linear') != -1:
		nn.init.kaiming_normal(lyr.weight.data, a=0, mode='fan_in')
	elif classname.find('BatchNorm') != -1:
		lyr.weight.data.normal_(mean=0, std=math.sqrt(2./9./64.)).\
			clamp_(-0.025, 0.025)
		nn.init.constant(lyr.bias.data, 0.0)

def batch_psnr(img, imclean, data_range):
	r"""
	Computes the PSNR along the batch dimension (not pixel-wise)

	Args:
		img: a `torch.Tensor` containing the restored image
		imclean: a `torch.Tensor` containing the reference image
		data_range: The data range of the input image (distance between
			minimum and maximum possible values). By default, this is estimated
			from the image data-type.
	"""
	img_cpu = img.data.cpu().numpy().astype(np.float32)
	imgclean = imclean.data.cpu().numpy().astype(np.float32)
	psnr = 0
	for i in range(img_cpu.shape[0]):
		psnr += peak_signal_noise_ratio(imgclean[i, :, :, :], img_cpu[i, :, :, :], \
					   data_range=data_range)
	return psnr/img_cpu.shape[0]

def data_augmentation(image, mode):
	r"""Performs dat augmentation of the input image

	Args:
		image: a cv2 (OpenCV) image
		mode: int. Choice of transformation to apply to the image
			0 - no transformation
			1 - flip up and down
			2 - rotate counterwise 90 degree
			3 - rotate 90 degree and flip up and down
			4 - rotate 180 degree
			5 - rotate 180 degree and flip
			6 - rotate 270 degree
			7 - rotate 270 degree and flip
	"""
	out = np.transpose(image, (1, 2, 0))
	if mode == 0:
		# original
		out = out
	elif mode == 1:
		# flip up and down
		out = np.flipud(out)
	elif mode == 2:
		# rotate counterwise 90 degree
		out = np.rot90(out)
	elif mode == 3:
		# rotate 90 degree and flip up and down
		out = np.rot90(out)
		out = np.flipud(out)
	elif mode == 4:
		# rotate 180 degree
		out = np.rot90(out, k=2)
	elif mode == 5:
		# rotate 180 degree and flip
		out = np.rot90(out, k=2)
		out = np.flipud(out)
	elif mode == 6:
		# rotate 270 degree
		out = np.rot90(out, k=3)
	elif mode == 7:
		# rotate 270 degree and flip
		out = np.rot90(out, k=3)
		out = np.flipud(out)
	else:
		raise Exception('Invalid choice of image transformation')
	return np.transpose(out, (2, 0, 1))

def variable_to_cv2_image(varim):
	r"""Converts a torch.autograd.Variable to an OpenCV image

	Args:
		varim: a torch.autograd.Variable
	"""
	nchannels = varim.size()[1]
	if nchannels == 1:
		res = (varim.data.cpu().numpy()[0, 0, :]*255.).clip(0, 255).astype(np.uint8)
	elif nchannels == 3:
		res = varim.data.cpu().numpy()[0]
		res = cv2.cvtColor(res.transpose(1, 2, 0), cv2.COLOR_RGB2BGR)
		res = (res*255.).clip(0, 255).astype(np.uint8)
	else:
		raise Exception('Number of color channels not supported')
	return res

def get_git_revision_short_hash():
	r"""Returns the current Git commit.
	"""
	return subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD']).strip()

def init_logger(argdict):
	r"""Initializes a logging.Logger to save all the running parameters to a
	log file

	Args:
		argdict: dictionary of parameters to be logged
	"""
	from os.path import join

	logger = logging.getLogger(__name__)
	logger.setLevel(level=logging.INFO)
	fh = logging.FileHandler(join(argdict.log_dir, 'log.txt'), mode='a')
	formatter = logging.Formatter('%(asctime)s - %(message)s')
	fh.setFormatter(formatter)
	logger.addHandler(fh)
	try:
		logger.info("Commit: {}".format(get_git_revision_short_hash()))
	except Exception as e:
		logger.error("Couldn't get commit number: {}".format(e))
	logger.info("Arguments: ")
	for k in argdict.__dict__:
		logger.info("\t{}: {}".format(k, argdict.__dict__[k]))

	return logger

def init_logger_ipol():
	r"""Initializes a logging.Logger in order to log the results after
	testing a model

	Args:
		result_dir: path to the folder with the denoising results
	"""
	logger = logging.getLogger('testlog')
	logger.setLevel(level=logging.INFO)
	fh = logging.FileHandler('out.txt', mode='w')
	formatter = logging.Formatter('%(message)s')
	fh.setFormatter(formatter)
	logger.addHandler(fh)

	return logger

def init_logger_test(result_dir):
	r"""Initializes a logging.Logger in order to log the results after testing
	a model

	Args:
		result_dir: path to the folder with the denoising results
	"""
	from os.path import join

	logger = logging.getLogger('testlog')
	logger.setLevel(level=logging.INFO)
	fh = logging.FileHandler(join(result_dir, 'log.txt'), mode='a')
	formatter = logging.Formatter('%(asctime)s - %(message)s')
	fh.setFormatter(formatter)
	logger.addHandler(fh)

	return logger

def normalize(data):
	r"""Normalizes a unit8 image to a float32 image in the range [0, 1]

	Args:
		data: a unint8 numpy array to normalize from [0, 255] to [0, 1]
	"""
	return np.float32(data/255.)

def svd_orthogonalization(lyr):
	r"""Applies regularization to the training by performing the
	orthogonalization technique described in the paper "FFDNet:	Toward a fast
	and flexible solution for CNN based image denoising." Zhang et al. (2017).
	For each Conv layer in the model, the method replaces the matrix whose columns
	are the filters of the layer by new filters which are orthogonal to each other.
	This is achieved by setting the singular values of a SVD decomposition to 1.

	This function is to be called by the torch.nn.Module.apply() method,
	which applies svd_orthogonalization() to every layer of the model.
	"""
	classname = lyr.__class__.__name__
	if classname.find('Conv') != -1:
		weights = lyr.weight.data.clone()
		c_out, c_in, f1, f2 = weights.size()
		dtype = lyr.weight.data.type()

		# Reshape filters to columns
		# From (c_out, c_in, f1, f2)  to (f1*f2*c_in, c_out)
		weights = weights.permute(2, 3, 1, 0).contiguous().view(f1*f2*c_in, c_out)

		# Convert filter matrix to numpy array
		weights = weights.cpu().numpy()

		# SVD decomposition and orthogonalization
		mat_u, _, mat_vh = np.linalg.svd(weights, full_matrices=False)
		weights = np.dot(mat_u, mat_vh)

		# As full_matrices=False we don't need to set s[:] = 1 and do mat_u*s
		lyr.weight.data = torch.Tensor(weights).view(f1, f2, c_in, c_out).\
			permute(3, 2, 0, 1).type(dtype)
	else:
		pass

def remove_dataparallel_wrapper(state_dict):
	r"""Converts a DataParallel model to a normal one by removing the "module."
	wrapper in the module dictionary

	Args:
		state_dict: a torch.nn.DataParallel state dictionary
	"""
	from collections import OrderedDict

	new_state_dict = OrderedDict()
	for k, vl in state_dict.items():
		name = k[7:] # remove 'module.' of DataParallel
		new_state_dict[name] = vl

	return new_state_dict

def is_rgb(im_path):
	r""" Returns True if the image in im_path is an RGB image
	"""
	from skimage.io import imread
	rgb = False
	im = imread(im_path)
	if (len(im.shape) == 3):
		if not(np.allclose(im[...,0], im[...,1]) and np.allclose(im[...,2], im[...,1])):
			rgb = True
	print("rgb: {}".format(rgb))
	print("im shape: {}".format(im.shape))
	return rgb


08/03/24

misc

In [12]:
!pip install easydict


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import torch
print(torch.__version__)

2.4.0+cu118
